## To cover all the concepts and commands of PySpark for Data Engineering.

In [0]:
from pyspark.sql import functions as F

### 1. PySpark Basics

**Starting a Spark Session:**

In [0]:
# Starting a Spark Session:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("App Name").getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

**Reading Data:**

In [0]:
df = spark.read.format("csv") \
          .option("header", "true") \
          .option("inferSchema", "true") \
          .load("dbfs:/FileStore/shared_uploads/zaderohish5@gmail.com/employee_data.csv")
df.show()

# you can give the other formats: parquet, json, orc etc

+-----------+------------+----------+------+------------+---------+
|employee_id|        name|department|salary|joining_date| location|
+-----------+------------+----------+------+------------+---------+
|        101|Rohan Sharma|        IT| 75000|  2020-05-12|Bangalore|
|        102|  Priya Iyer|        HR| 65000|  2019-08-25|    Delhi|
|        103|Rajesh Kumar|   Finance| 80000|  2021-03-15|   Mumbai|
|        104| Sneha Patil|        IT| 78000|  2018-07-30|     Pune|
|        105| Amit Sharma| Marketing| 72000|  2022-01-10|Hyderabad|
|        106|  Ananya Das|        HR| 67000|  2017-11-20|  Kolkata|
|        107|Vikram Singh|   Finance| 85000|  2023-06-05|  Chennai|
|        108| Rohit Verma|        IT| 76000|  2020-09-18|Bangalore|
|        109| Arjun Mehta| Marketing| 73000|  2019-12-11|    Delhi|
|        110| Rohish Zade|   Finance| 81000|  2016-04-22|   Mumbai|
+-----------+------------+----------+------+------------+---------+



In [0]:
# CSV
df = spark.read.csv("path_to_file.csv", header=True, inferSchema=True)

# Parquet
df = spark.read.parquet("path_to_file.parquet")

# JSON
df = spark.read.json("path_to_file.json")


**Writing Data:**

In [0]:
df.write.csv("output_path.csv", header=True)
df.write.parquet("output_path.parquet")
df.write.json("output_path.json")

### 2. DataFrame Operations

**Show data:**

In [0]:
# show first 5 rows
df.show(5)

+-----------+------------+----------+------+------------+---------+
|employee_id|        name|department|salary|joining_date| location|
+-----------+------------+----------+------+------------+---------+
|        101|Rohan Sharma|        IT| 75000|  2020-05-12|Bangalore|
|        102|  Priya Iyer|        HR| 65000|  2019-08-25|    Delhi|
|        103|Rajesh Kumar|   Finance| 80000|  2021-03-15|   Mumbai|
|        104| Sneha Patil|        IT| 78000|  2018-07-30|     Pune|
|        105| Amit Sharma| Marketing| 72000|  2022-01-10|Hyderabad|
+-----------+------------+----------+------+------------+---------+
only showing top 5 rows



In [0]:
# prints schema
df.printSchema()

root
 |-- employee_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- joining_date: date (nullable = true)
 |-- location: string (nullable = true)



In [0]:
# list the the columns
df.columns

Out[7]: ['employee_id', 'name', 'department', 'salary', 'joining_date', 'location']

In [0]:
# summary statistics
df.describe().show()

+-------+------------------+------------+----------+-----------------+---------+
|summary|       employee_id|        name|department|           salary| location|
+-------+------------------+------------+----------+-----------------+---------+
|  count|                10|          10|        10|               10|       10|
|   mean|             105.5|        null|      null|          75200.0|     null|
| stddev|3.0276503540974917|        null|      null|6214.677966091424|     null|
|    min|               101| Amit Sharma|   Finance|            65000|Bangalore|
|    max|               110|Vikram Singh| Marketing|            85000|     Pune|
+-------+------------------+------------+----------+-----------------+---------+



**Select columns:**

In [0]:
df.select("employee_id", "name").show()

+-----------+------------+
|employee_id|        name|
+-----------+------------+
|        101|Rohan Sharma|
|        102|  Priya Iyer|
|        103|Rajesh Kumar|
|        104| Sneha Patil|
|        105| Amit Sharma|
|        106|  Ananya Das|
|        107|Vikram Singh|
|        108| Rohit Verma|
|        109| Arjun Mehta|
|        110| Rohish Zade|
+-----------+------------+



**Filter data:**

In [0]:
df.filter(df["employee_id"] == 105).show()

+-----------+-----------+----------+------+------------+---------+
|employee_id|       name|department|salary|joining_date| location|
+-----------+-----------+----------+------+------------+---------+
|        105|Amit Sharma| Marketing| 72000|  2022-01-10|Hyderabad|
+-----------+-----------+----------+------+------------+---------+



In [0]:
df.filter(df["name"] == 'Rohish Zade').show()

+-----------+-----------+----------+------+------------+--------+
|employee_id|       name|department|salary|joining_date|location|
+-----------+-----------+----------+------+------------+--------+
|        110|Rohish Zade|   Finance| 81000|  2016-04-22|  Mumbai|
+-----------+-----------+----------+------+------------+--------+



**Group By and Aggregation:**

In [0]:
df.groupBy("department").agg(F.sum("salary").alias("total_salary")).show()

+----------+------------+
|department|total_salary|
+----------+------------+
|        HR|      132000|
|   Finance|      246000|
| Marketing|      145000|
|        IT|      229000|
+----------+------------+



### 3. Transformations in PySpark

**Map and FlatMap:**

- In PySpark, both map() and flatMap() are transformations that operate on each element of a Resilient Distributed Dataset (RDD) or each row of a DataFrame. 
- They apply a function you provide to each element/row and return a new RDD/DataFrame. 
- The key difference lies in how they handle the return values of your function:

**map():**
- `One-to-one transformation:` For each element in the input RDD/DataFrame, the map() function applies your provided function and returns exactly one element in the new RDD/DataFrame.
- `Preserves structure (to a degree):` If your function returns a collection (like a list), that entire collection is treated as a single element in the resulting RDD/DataFrame.

In [0]:
data = ["hello world", "hi there", "spark"]
rdd = spark.sparkContext.parallelize(data)

def split_string(s):
    return s.split(" ")

mapped_rdd = rdd.map(split_string)
print(mapped_rdd.collect())

# In this example, map() applied split_string to each string, and each resulting list of words became a single element in the mapped_rdd.

[['hello', 'world'], ['hi', 'there'], ['spark']]


**flatMap():**

- `One-to-many (or one-to-zero) transformation followed by flattening:` The flatMap() function first applies your provided function to each element. However, the function you provide to flatMap() is expected to return an iterable (like a list, tuple, or sequence) for each input element. flatMap() then flattens the resulting collection of iterables into a single RDD/DataFrame.
- `Changes structure:` The number of elements in the resulting RDD/DataFrame from flatMap() can be different from the original. If your function returns a list of multiple items for one input element, those multiple items will become individual elements in the output. If your function returns an empty iterable, that input element will effectively be removed from the output.

In [0]:
data = ["hello world", "hi there", "spark"]
rdd = spark.sparkContext.parallelize(data)

def split_string_flat(s):
    return s.split(" ")

flat_mapped_rdd = rdd.flatMap(split_string_flat)
print(flat_mapped_rdd.collect())

# Here, flatMap() also applied the splitting function, but then it flattened the resulting lists, so the flat_mapped_rdd contains individual words.

['hello', 'world', 'hi', 'there', 'spark']
